In [ ]:
#
#
# Universidad EAFIT
# 2024-2
#

In [ ]:
#configuración en google colab de spark y pyspark
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

data = pd.read_csv('/content/medellin_clima_2_anios.csv')

data['fecha_hora'] = pd.to_datetime(data['fecha_hora'])
data = data.sort_values(['codigo_x', 'fecha_hora']).reset_index(drop=True)

data['hora'] = data['fecha_hora'].dt.hour
data['minuto'] = data['fecha_hora'].dt.minute
data['dia_semana'] = data['fecha_hora'].dt.dayofweek

data['hora_sin'] = np.sin(2 * np.pi * data['hora'] / 24)
data['hora_cos'] = np.cos(2 * np.pi * data['hora'] / 24)
data['p_sum_t-1'] = data.groupby('codigo_x')['p_sum'].shift(1)
data['p_sum_t-2'] = data.groupby('codigo_x')['p_sum'].shift(2)
data['p_sum_t-3'] = data.groupby('codigo_x')['p_sum'].shift(3)
data['p_sum_t-4'] = data.groupby('codigo_x')['p_sum'].shift(4)
data['nivel_metros_promedio_pasado'] = data.groupby('codigo_x')['nivel_metros_promedio'].shift(1)
data['p_sum_acumulado_1h'] = data['p_sum'].shift(1).rolling(window=10).sum()

data['nivel_metros_futuro'] = data.groupby('codigo_x')['nivel_metros_promedio'].shift(-1)

data

,codigo_x,fecha_hora,h_promedio,h_std,h_sum,t_promedio,t_std,t_sum,pr_promedio,pr_std,...,dia_semana,hora_sin,hora_cos,p_sum_t-1,p_sum_t-2,p_sum_t-3,p_sum_t-4,nivel_metros_promedio_pasado,p_sum_acumulado_1h,nivel_metros_futuro
0,197,2022-06-29 14:45:00,60.322667,0.593506,904.84,24.714667,0.105076,370.72,842.491333,0.065232,...,2,-0.500000,-0.866025,NaN,NaN,NaN,NaN,NaN,NaN,0.596887
1,197,2022-06-29 15:00:00,59.664667,1.366768,894.97,24.759333,0.070353,371.39,842.331333,0.040685,...,2,-0.707107,-0.707107,0.0,NaN,NaN,NaN,0.597520,NaN,0.590020
2,197,2022-06-29 15:15:00,59.628000,0.709368,894.42,24.617333,0.112787,369.26,842.247333,0.075068,...,2,-0.707107,-0.707107,0.0,0.0,NaN,NaN,0.596887,NaN,0.584540
3,197,2022-06-29 15:30:00,59.266667,0.651050,889.00,24.583333,0.106413,368.75,842.081333,0.064128,...,2,-0.707107,-0.707107,0.0,0.0,0.0,NaN,0.590020,NaN,0.581833
4,197,2022-06-29 15:45:00,61.250000,0.883338,918.75,24.980000,0.165357,374.70,841.872000,0.065049,...,2,-0.707107,-0.707107,0.0,0.0,0.0,0.0,0.584540,NaN,0.578520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14832,197,2022-12-01 02:45:00,93.686667,0.299754,1405.30,16.549333,0.045429,248.24,842.174000,0.056290,...,3,0.500000,0.866025,0.0,0.0,0.0,0.0,0.585532,0.0,0.584784
14833,197,2022-12-01 03:00:00,93.558667,0.456428,1403.38,16.582667,0.038260,248.74,842.262667,0.119968,...,3,0.707107,0.707107,0.0,0.0,0.0,0.0,0.585158,0.0,0.584410
14834,197,2022-12-01 03:15:00,93.580000,0.459021,1403.70,16.600000,0.000000,249.00,842.494667,0.046425,...,3,0.707107,0.707107,0.0,0.0,0.0,0.0,0.584784,0.0,0.584036
14835,197,2022-12-01 03:30:00,93.531333,0.464756,1402.97,16.499333,0.072256,247.49,842.548000,0.047839,...,3,0.707107,0.707107,0.0,0.0,0.0,0.0,0.584410,0.0,0.583662


In [ ]:
data_cleaned = data.dropna(subset=['nivel_metros_futuro', 'p_sum', 'nivel_metros_promedio'])

features = ['p_sum', 'hora', 'p_sum_t-1','p_sum_t-2','p_sum_t-3','p_sum_t-4','nivel_metros_promedio_pasado']
X = data_cleaned[features]
y = data_cleaned['nivel_metros_futuro']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

rf_regressor = RandomForestRegressor(random_state=42, n_estimators=100)
rf_regressor.fit(X_train, y_train)

y_pred = rf_regressor.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"MAE: {mae:.2f}, RMSE: {rmse:.2f}, R2: {r2:.2f}")

MAE: 0.03, RMSE: 0.09, R2: 0.71


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
